# Gradient Boosting Classifier

A continuación se entrena un modelo de Gradient Boosting Classification para predecir el conjunto de datos del Titanic.

## Limpieza de Datos

In [2]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Importar archivos

In [3]:
train_data= pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/titanic/train.csv")
df_test = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/titanic/test.csv")

### Identificación de datos nulos


In [4]:
 # Contar los valores nulos en cada columna
null_counts = train_data.isnull().sum()

print(null_counts)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


En porcentaje:

In [5]:
#Obtenemos lo mismo pero con porcentajes
null_percentages = (train_data.isnull().sum() / len(train_data)) * 100
print(null_percentages)

PassengerId     0.000000
Survived        0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            19.865320
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.000000
Cabin          77.104377
Embarked        0.224467
dtype: float64


### Cambio de datos nulos en *AGE*
El objetivo de esta porción de código es extraer los títulos de las personas de la columna "Name" en el conjunto de datos. Los nombres de las personas incluyen títulos como "Mr.", "Mrs.", "Dr.", etc. Extraemos estos titulo, se meten en grupos y se cada el promedio para cada uno, luego si existe una edad faltante para un pasajero se encuentra esta relacion y se pone la media del grupo al que pertenece.


In [6]:
# Primero extraemos los titulos de de la columna Name
train_data['Title'] = train_data['Name'].apply(lambda x: re.search(' ([A-Za-z]+)\.', x).group(1))

df_test['Title'] = df_test['Name'].apply(lambda x: re.search(' ([A-Za-z]+)\.', x).group(1))
#La funcion  (=re.search(' ([A-Za-z]+)\.', x) encuentra la relacion de un espacio, luego uno o más caracteres alfabéticos en mayúsculas o minúsculas, seguidos de un punto.

# Calculamos la media para data titulo
median_age_by_title = train_data.groupby('Title')['Age'].median()

# Ponemos las edades en donde falten
for title, median_age in median_age_by_title.items():
    train_data.loc[(train_data['Age'].isnull()) & (train_data['Title'] == title), 'Age'] = median_age

for title, median_age in median_age_by_title.items():
    df_test.loc[(df_test['Age'].isnull()) & (df_test['Title'] == title), 'Age'] = median_age

In [7]:
#Imprimimos la edad media de cada grupo
print(median_age_by_title)

Title
Capt        70.0
Col         58.0
Countess    33.0
Don         40.0
Dr          46.5
Jonkheer    38.0
Lady        48.0
Major       48.5
Master       3.5
Miss        21.0
Mlle        24.0
Mme         24.0
Mr          30.0
Mrs         35.0
Ms          28.0
Rev         46.5
Sir         49.0
Name: Age, dtype: float64


### Sustitucion de Sexo y cabino por valores binarios
En el archivo de analisis exploratorio para poder hacer la matriz de correlacion, se necesito aplicar el cambio de estas variables a numericos,y vemos que esto es posible debido a que podemos decir que entre sexos hay prioridad ya que las mujeres tienen muchas mas posibilidades de sobrevivir, y  para la cabina se ve si hay cabina asignada o no la hay.

In [8]:
median_age_by_title = train_data.groupby('Pclass')['Fare'].median()
median_age_by_title

Pclass
1    60.2875
2    14.2500
3     8.0500
Name: Fare, dtype: float64

In [9]:
df_test.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
Title            0
dtype: int64

In [10]:
train_data['Sex'] = train_data['Sex'].map({"male":1,
"female":0})
train_data['Cabin'] = np.where(train_data['Cabin'].isnull(), 0, 1)

df_test['Sex'] = df_test['Sex'].map({"male":1,
"female":0})

df_test['Cabin'] = np.where(df_test['Cabin'].isnull(), 0, 1)

In [11]:
df_test[df_test['Fare'].isna()]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
152,1044,3,"Storey, Mr. Thomas",1,60.5,0,0,3701,NaN,0,S,Mr


In [12]:
df_test = df_test.fillna({'Fare': 8.0500})

In [13]:
 # Contamos los valores nulos en la columna de edad en nuestro neuvo data frame
null_counts = train_data.isnull().sum()

print(null_counts)

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       2
Title          0
dtype: int64


Remplazo de Embarked por los valores con mas frecuencia en el dataset

In [14]:
train_data["Embarked"].fillna("S", inplace = True)
null_counts = train_data.isnull().sum()

print(null_counts)

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
Title          0
dtype: int64


In [15]:
train_data['Embarked_S'] = np.where(train_data['Embarked']=='S',1,0)
train_data['Embarked_Q'] = np.where(train_data['Embarked']=='Q',1,0)
train_data['Embarked_C'] = np.where(train_data['Embarked']=='C',1,0)

df_test['Embarked_S'] = np.where(df_test['Embarked']=='S',1,0)
df_test['Embarked_Q'] = np.where(df_test['Embarked']=='Q',1,0)
df_test['Embarked_C'] = np.where(df_test['Embarked']=='C',1,0)

Eliminar columnas no numéricas

In [16]:
y_train = train_data['Survived']
drop_cols = ['PassengerId', 'Survived', 'Ticket', 'Embarked', 'Title', 'Name']
X_train= train_data.drop(drop_cols, axis = 1)
y_train = [y_train]

In [17]:
y_train =np.array(y_train) # Convertir y_train a array
y_train = y_train.ravel() # Invertimos de (1, 891) a (891, 1)

## Modelo de aprendizaje automático: Gradient Boosting Classifier
A continuación se entrena el modelo de aprendizaje automático. Se optó en este caso por utilizar un modelo de potenciación de gradiente.

In [18]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns

# Crear el modelo de GBM
gbm_model = GradientBoostingClassifier()

# Entrenar el modelo
gbm_model.fit(X_train, y_train)


GradientBoostingClassifier()

---

### Preparar datos de prueba

In [20]:
drop_cols = ['PassengerId', 'Ticket', 'Embarked', 'Title', 'Name']
output_df = df_test
df_test = df_test.drop(drop_cols, axis = 1)

## Realizamos predicciones en el conjunto de datos de prueba con el modelo entrenado anteriormente

In [21]:
# Realizar predicciones en el conjunto de prueba
y_pred = gbm_model.predict(df_test)
y_pred.tofile('output.csv', sep = '')
output_df['Survived'] = y_pred
drop_cols = ['Pclass',	'Name'	,'Sex'	,'Age'	,'SibSp'	,'Parch'	,'Ticket',	'Fare'	,'Cabin'	,'Embarked'	,'Title',	'Embarked_S',	'Embarked_Q' ,'Embarked_C']
output_df = output_df.drop(drop_cols, axis = 1)
output_df.to_csv('output.csv', index = False)

Obtener archivo de salida con resultados de predicciones en el conjunto de datos de prueba